In [11]:
import joblib
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error
from scipy.optimize import minimize
from sklearn.linear_model import LinearRegression

In [12]:
file_path = r"C:\Users\advait\Desktop\converted_distillation_data.csv"
df = pd.read_csv(file_path)
inputs = ['Feed_Flow_Rate', 'Feed_Composition', 'Reflux_Ratio', 'Boil_Up_Ratio']
X = df[inputs]

In [13]:
rf_model = joblib.load('rf_model.pkl')
xgb_model = joblib.load('xgb_model.pkl')

In [14]:
X_test = pd.read_csv('X_test.csv')
y_test = pd.read_csv('y_test.csv')
X_train=pd.read_csv('X_train.csv')
y_train=pd.read_csv('y_train.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'X_train.csv'

In [ ]:
train_pred1 = rf_model.predict(X_train)
train_pred2 = xgb_model.predict(X_train)

In [ ]:
train_meta = np.column_stack((train_pred1, train_pred2))
meta_model = LinearRegression()
meta_model.fit(train_meta, y_train)

In [ ]:
def predict_from_user_input():

    feed_flow_rate = float(input("Enter Feed Flow Rate: "))
    feed_composition = float(input("Enter Feed Composition: "))
    reflux_ratio = float(input("Enter Reflux Ratio: "))
    boil_up_ratio = float(input("Enter Boil-Up Ratio: "))

    user_input = pd.DataFrame({
        'Feed_Flow_Rate': [feed_flow_rate],
        'Feed_Composition': [feed_composition],
        'Reflux_Ratio': [reflux_ratio],
        'Boil_Up_Ratio': [boil_up_ratio]
    })

    pred1 = rf_model.predict(user_input)
    pred2 = xgb_model.predict(user_input) 

    meta_input = np.column_stack((pred1, pred2))

    final_prediction = meta_model.predict(meta_input)

    print("Predicted Output:", final_prediction)

predict_from_user_input()